<a href="https://colab.research.google.com/github/XeniaZhou/Pacman-in-Gym/blob/master/Rendering_OpenAi_Gym_in_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#          _____                _____                    _____                    _____                    _____                    _____          
#         /\    \              /\    \                  /\    \                  /\    \                  /\    \                  /\    \         
#        /::\    \            /::\    \                /::\    \                /::\    \                /::\    \                /::\    \        
#       /::::\    \           \:::\    \              /::::\    \              /::::\    \              /::::\    \               \:::\    \       
#      /::::::\    \           \:::\    \            /::::::\    \            /::::::\    \            /::::::\    \               \:::\    \      
#     /:::/\:::\    \           \:::\    \          /:::/\:::\    \          /:::/\:::\    \          /:::/\:::\    \               \:::\    \     
#    /:::/__\:::\    \           \:::\    \        /:::/__\:::\    \        /:::/__\:::\    \        /:::/__\:::\    \               \:::\    \    
#    \:::\   \:::\    \          /::::\    \      /::::\   \:::\    \      /::::\   \:::\    \      /::::\   \:::\    \              /::::\    \   
#  ___\:::\   \:::\    \        /::::::\    \    /::::::\   \:::\    \    /::::::\   \:::\    \    /::::::\   \:::\    \    ____    /::::::\    \  
# /\   \:::\   \:::\    \      /:::/\:::\    \  /:::/\:::\   \:::\    \  /:::/\:::\   \:::\____\  /:::/\:::\   \:::\    \  /\   \  /:::/\:::\    \ 
#/::\   \:::\   \:::\____\    /:::/  \:::\____\/:::/  \:::\   \:::\____\/:::/  \:::\   \:::|    |/:::/  \:::\   \:::\____\/::\   \/:::/  \:::\____\
#\:::\   \:::\   \::/    /   /:::/    \::/    /\::/    \:::\  /:::/    /\::/   |::::\  /:::|____|\::/    \:::\  /:::/    /\:::\  /:::/    \::/    /
# \:::\   \:::\   \/____/   /:::/    / \/____/  \/____/ \:::\/:::/    /  \/____|:::::\/:::/    /  \/____/ \:::\/:::/    /  \:::\/:::/    / \/____/ 
#  \:::\   \:::\    \      /:::/    /                    \::::::/    /         |:::::::::/    /            \::::::/    /    \::::::/    /          
#   \:::\   \:::\____\    /:::/    /                      \::::/    /          |::|\::::/    /              \::::/    /      \::::/____/           
#    \:::\  /:::/    /    \::/    /                       /:::/    /           |::| \::/____/               /:::/    /        \:::\    \           
#     \:::\/:::/    /      \/____/                       /:::/    /            |::|  ~|                    /:::/    /          \:::\    \          
#      \::::::/    /                                    /:::/    /             |::|   |                   /:::/    /            \:::\    \         
#       \::::/    /                                    /:::/    /              \::|   |                  /:::/    /              \:::\____\        
#        \::/    /                                     \::/    /                \:|   |                  \::/    /                \::/    /        
#         \/____/                                       \/____/                  \|___|                   \/____/                  \/____/    

# install dependancies, takes around 45 seconds

Rendering Dependancies



In [ ]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Pacman Dependancies

In [ ]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

# Imports and Helper functions


In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [ ]:
!pip install tensorflow==1.4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# Pacman!

In [ ]:
env = gym.make("MsPacman-v0")

In [ ]:
#check out the pacman action space!
print(env.action_space)

In [ ]:
from tensorflow.contrib.layers import flatten, conv2d, fully_connected
from collections import deque, Counter
import random
from datetime import datetime

In [ ]:
color = np.array([210, 164, 74]).mean()
def preprocess_observation(obs):
  # Crop and resize the image
  img = obs[1:176:2, ::2]
  # Convert the image to greyscale
  img = img.mean(axis=2)
  # Improve image contrast
  img[img==color] = 0
  # Next we normalize the image from -1 to +1
  img =(img-128)/128-1
  return img.reshape(88,80,1)

In [ ]:
n_outputs = env.action_space.n

In [ ]:
def q_network(X, name_scope):
# Initialize layers
  initializer = tf.keras.initializers.VarianceScaling(scale=2.0)
  with tf.variable_scope(name_scope) as scope:
    # initialize the convolutional layers
    layer_1 = conv2d(X, num_outputs=32, kernel_size=(8,8), stride=4, padding='SAME', weights_initializer=initializer)

    layer_2 = conv2d(layer_1, num_outputs=64, kernel_size=(4,4),    stride=2, padding='SAME', weights_initializer=initializer)

    layer_3 = conv2d(layer_2, num_outputs=64, kernel_size=(3,3), stride=1, padding='SAME', weights_initializer=initializer)

    flat = flatten(layer_3)
    fc = fully_connected(flat, num_outputs=128, weights_initializer=initializer)
   
    #Add final output layer
    output = fully_connected(fc, num_outputs=n_outputs, activation_fn=None, weights_initializer=initializer)
    vars = {v.name[len(scope.name):]: v for v in tf.get_collection(key=tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope.name)}
    #Return both variables and outputs together
  return vars, output

In [ ]:
num_episodes = 800
batch_size = 48
input_shape = (None, 88, 80, 1)
#Recall shape is img.reshape(88,80,1)
learning_rate = 0.001
X_shape = (None, 88, 80, 1)
discount_factor = 0.97
global_step = 0
copy_steps = 100
steps_train = 4
start_steps = 2000

In [ ]:
epsilon = 0.5
eps_min = 0.05
eps_max = 1.0
eps_decay_steps = 500000
#
def epsilon_greedy(action, step):
  p = np.random.random(1).squeeze() #1D entries returned using squeeze
  epsilon = max(eps_min, eps_max -(eps_max-eps_min)*step/eps_decay_steps) #Decaying policy with more steps
  if np.random.rand() < epsilon:
    return np.random.randint(n_outputs)
  else:
    return action

In [ ]:
buffer_len = 20000
#Buffer is made from a deque — double ended queue
exp_buffer = deque(maxlen=buffer_len)
def sample_memories(batch_size):
  perm_batch = np.random.permutation(len(exp_buffer))[:batch_size]
  mem = np.array(exp_buffer)[perm_batch]
  return mem[:,0], mem[:,1], mem[:,2], mem[:,3], mem[:,4]

In [ ]:
# we build our Q network, which takes the input X and generates Q values for all the actions in the state
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=[None, 88, 80,1])    
print(X.name)   
mainQ, mainQ_outputs = q_network(X, 'mainQ')
# similarly we build our target Q network, for policy evaluation
targetQ, targetQ_outputs = q_network(X, 'targetQ')
copy_op = [tf.assign(main_name, targetQ[var_name]) for var_name, main_name in mainQ.items()]
copy_target_to_main = tf.group(*copy_op)

In [ ]:
X_action = tf.placeholder(tf.int32, shape=[None]) 
print(X.name)
Q_action = tf.reduce_sum(targetQ_outputs*tf.one_hot(X_action, 9), axis=1, keep_dims=True)
# define a placeholder for our output i.e action
y = tf.placeholder(tf.float32, shape=(None,1))
# now we calculate the loss which is the difference between actual value and predicted value
loss = tf.reduce_mean(input_tensor=tf.square(y - Q_action))
# we use adam optimizer for minimizing the loss
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)
init = tf.global_variables_initializer()
loss_summary = tf.summary.scalar('LOSS', loss)
merge_summary = tf.summary.merge_all()
file_writer = tf.summary.FileWriter("log.txt", tf.get_default_graph())
X=tf.get_default_graph().get_tensor_by_name("Placeholder:0")

In [ ]:
with tf.Session() as sess:
  init.run()
  # for each episode
  history = []
  for i in range(500):
    done = False
    obs = env.reset()
    epoch = 0
    episodic_reward = 0
    actions_counter = Counter()
    episodic_loss = []
    # while the state is not the terminal state
    while not done:
    # get the preprocessed game screen
      obs = preprocess_observation(obs)
    # feed the game screen and get the Q values for each action, 
      actions = mainQ_outputs.eval(feed_dict={X:[obs]})
    # get the action
      action = np.argmax(actions, axis=-1)
      actions_counter[str(action)] += 1
    # select the action using epsilon greedy policy
      action = epsilon_greedy(action, global_step)
    # now perform the action and move to the next state, next_obs, receive reward
      next_obs, reward, done, _ = env.step(action)
    # Store this transition as an experience in the replay buffer! Quite important
      exp_buffer.append([obs, action, preprocess_observation(next_obs), reward, done])
    # After certain steps we move on to generating y-values for Q network with samples from the experience replay buffer
      if global_step % steps_train == 0 and global_step > start_steps:
        o_obs, o_act, o_next_obs, o_rew, o_done = sample_memories(batch_size)
        # states
        o_obs = [x for x in o_obs]
        # next states
        o_next_obs = [x for x in o_next_obs]
        # next actions
        next_act = mainQ_outputs.eval(feed_dict={X:o_next_obs})
        #discounted reward for action: these are our Y-values
        y_batch = o_rew + discount_factor * np.max(next_act, axis=-1) * (1-o_done)
        # merge all summaries and write to the file
        mrg_summary = merge_summary.eval(feed_dict={X:o_obs, y:np.expand_dims(y_batch, axis=-1), X_action:o_act})
        file_writer.add_summary(mrg_summary, global_step)
        # To calculate the loss, we run the previously defined functions mentioned while feeding inputs
        train_loss, _ = sess.run([loss, training_op], feed_dict={X:o_obs, y:np.expand_dims(y_batch, axis=-1), X_action:o_act})
        episodic_loss.append(train_loss)
  # after some interval we copy our main Q network weights to target Q network
      if (global_step+1) % copy_steps == 0 and global_step > start_steps:
        copy_target_to_main.run()
      obs = next_obs
      epoch += 1
      global_step += 1
      episodic_reward += reward
    history.append(episodic_reward)
    print('Epochs per episode:', epoch, 'Episode Reward:', episodic_reward,"Episode number:", i, flush=True)

In [ ]:
#Evaluate model on openAi GYM
env = wrap_env(env)
observation = env.reset()
frames=[]
new_observation = observation
prev_input = None
done = False
with tf.Session() as sess:
  init.run()
  while True:
    if True:
    #set input to network to be difference image
      obs = preprocess_observation(observation)
    # feed the game screen and get the Q values for each action
      actions = mainQ_outputs.eval(feed_dict={X:[obs]})
    # get the action
      action = np.argmax(actions, axis=-1)
      actions_counter[str(action)] += 1
    # select the action using epsilon greedy policy
      action = epsilon_greedy(action, global_step)
      frames.append(env.render(mode = 'rgb_array'))
      observation = new_observation
    # now perform the action and move to the next state, next_obs, receive reward
      new_observation, reward, done, _ = env.step(action)
      if done:
      #observation = env.reset()
        break
  env.close()
  show_video()

In [ ]:
def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    display(display_animation(anim, default_mode='loop'))

display_frames_as_gif(frames)